In [ ]:
%matplotlib inline
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import date
import pandas as pd
import calendar
import numpy.ma as ma

import createFolder as cF
import plot_vertical as pvert
import plot_sfc_spaghetti as spagh
import fill_values as fv


import os




In [ ]:
# 16th doesnt exist, 17-19 have different shapes
t = [
#    1, 2, 3, 4, 5, 6 , 7, 8, 9,10, \
 #   11,12,13,14,15,    
  #  17,18,19,20, \
    21#,22,  
   # 23, \
#    24,25,26,27,28,29,30, \
 #   31
    ]
year = 2016
month = 12
tid = '00'
savefig = 1  # 1 = yes, 0 = no

station = 'Haukeliseter'

In [ ]:
var_name = 'snowfall_amount'
#var_name = 'rainfall_amount'
#var_name = 'graupelfall_amount'
#var_name = 'atmosphere_cloud_condensed_water_content'
#var_name = 'atmosphere_cloud_ice_content'
#var_name = 'air_temperature'
var_name_ml = '%s_ml' %(var_name)


ncdir_ml = '../../MEPS_data/%s/ml/%s' %(station,var_name_ml)
figdir = '../../MEPS_fig/%s' %(var_name_ml)
form = 'png'
cF.createFolder('%s' %(figdir))

In [ ]:
for day in t:
    if day < 10:
        day = '0%s' %(str(day))


## open netCDF 
    fn_ml = dict()
    time_ml = dict()
    lead_time = dict()
    for ens_memb  in range(0,10):
        fn_ml[ens_memb] = netCDF4.Dataset('%s/%s%s%s_%s_%s.nc' %(ncdir_ml,year,month,day,tid,ens_memb))
# Read in time to know initialisation time (plotting)    
        time_ml[ens_memb] = fn_ml[ens_memb].variables['time']

    ini_day = datetime.datetime.utcfromtimestamp(time_ml[0][0]).day   # day of initialisation
    hh = datetime.datetime.utcfromtimestamp(time_ml[0][0]).hour       # first hour of initialisation? 
    my_date = date(year,month,ini_day)
    calday = calendar.day_name[my_date.weekday()]
    calmon = calendar.month_abbr[int(month)]
    
    
    
    snow_amount_ml = dict()
    thickness_ml = dict()
    h = dict()
    td = dict()
    height = dict()
    time = dict()
    for ens_memb in range(0,10):
# Read in the variable name which should be plotted  
        snow_amount_ml[ens_memb] = fn_ml[ens_memb].variables[var_name_ml]
# Read in the thickness of a layer
        thickness_ml[ens_memb] = fn_ml[ens_memb].variables['layer_thickness']
# create an array with time of the shape of the variable
        lead_time = np.arange(0,time_ml[0].size)
        model_level = np.arange(0,snow_amount_ml[ens_memb].shape[1])
        td,Y = np.meshgrid(lead_time,model_level)

# substitute missing values with nan       
        snow_amount_ml[ens_memb] = fv.fill_nan(snow_amount_ml[ens_memb][:])
        thickness_ml[ens_memb] = fv.fill_nan(thickness_ml[ens_memb][:])

# calculate the height from the thickness
        thickness = thickness_ml[ens_memb]
        h_above = []
        h_above.append(np.zeros(thickness[:,:].shape[0]))
        
        th_arr = np.nansum([ thickness[:,0], thickness[:,(0+1)]], axis = 0)
        h_above.append(th_arr)
        for i in range(2,thickness[:,:].shape[1]):
            th_arr = np.nansum([h_above[i-1], thickness[:,i]], axis = 0)
            h_above.append(th_arr)
        h_above = np.transpose(h_above)
# get only valuable values not nan    
        time[ens_memb] =(td[:,~np.isnan(snow_amount_ml[ens_memb]).any(axis=1)])
        height[ens_memb] = (h_above[~np.isnan(snow_amount_ml[ens_memb]).any(axis=1)])
        thickness_ml[ens_memb] = (thickness_ml[ens_memb][~np.isnan(snow_amount_ml[ens_memb]).any(axis=1),:])
        snow_amount_ml[ens_memb] = (snow_amount_ml[ens_memb][~np.isnan(snow_amount_ml[ens_memb]).any(axis=1)])
# convert the snow amount from [kg/m^2] into [g/m^2] by multiply with 1000
        snow_amount_ml[ens_memb] = (snow_amount_ml[ens_memb]*1000.)
#        snow_amount_ml[ens_memb] = (snow_amount_ml[ens_memb]*1000.)/thickness_ml[ens_memb]

In [ ]:
maxim = []
for ens_memb in range(0,10):
    maxi = snow_amount_ml[ens_memb].max()
    maxim.append(maxi)
maxim = np.asarray(maxim)
np.nanmax(maxim)

In [ ]:
var_name_ml = var_name_ml.split("_")

In [ ]:

unit = '[g/m$^3$]'
fig_name = '%s%s%s_0%s.png' %(year,month,ini_day,hh)
title = '%s, %s %s %s 0%s UTC' %( calday,ini_day,calmon,year,hh)

pvert.plot_vertical_EM(time, height,snow_amount_ml,var_name_ml, unit, fig_name, title, savefig, figdir, fig_name, form)
plt.close()

In [ ]:
fig_name = '%s%s%s_0%s_48h.png' %(year,month,ini_day,hh)
pvert.plot_vertical_EM_48h(time, height,snow_amount_ml,var_name_ml, unit, fig_name, title, savefig, figdir, fig_name, form)
plt.close()

In [ ]:
fn_ml.close()